In [9]:
from datetime import datetime, timedelta
from json import loads, JSONDecodeError
from logging import basicConfig, CRITICAL ,ERROR, getLogger, INFO, log
from os import _exit, getenv, makedirs, path
from re import findall
from time import localtime, sleep, strftime, time
from traceback import TracebackException

from dotenv import load_dotenv
from openpyxl import load_workbook
from pandas import DataFrame
from seleniumwire import webdriver
from seleniumwire.utils import decode
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.remote_connection import LOGGER as seleniumLogger
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from urllib3.connectionpool import log as urllibLogger
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
class Errores:
    def __init__(self):
        self._errores = {
            "Clase": [],
            "Mensaje": [],
            "Linea de Error": [],
            "Codigo Error": [],
            "Publicacion": []
        }
        
    def _get_errores(self):
        return self._errores
    
    def _append_error(self, error, enlace):
        traceback_error = TracebackException.from_exception(error)
        error_message = traceback_error._str
        error_stack = traceback_error.stack[0]
        log(ERROR, error_message)  
        self._errores["Clase"].append(traceback_error.exc_type)
        self._errores["Mensaje"].append(error_message)
        self._errores["Linea de Error"].append(error_stack.lineno)
        self._errores["Codigo Error"].append(error_stack.line)
        self._errores["Publicacion"].append(enlace)

In [3]:
class Dataset:
    def __init__(self):
        self._dataset = {
            "Fecha Extraccion": [],
            "titulo_marketplace": [],
            "tiempo_creacion": [],
            "tipo_delivery": [],
            "delivery_data": [],
            "delivery_direccion": [],
            "descripcion": [],
            "disponible": [],
            "vendido": [],
            "fecha_union_vendedor": [],
            "cantidad": [],
            "precio": [],
            "tipo_moneda": [],
            "amount_with_concurrency": [],
            "latitud": [],
            "longitud": [],
            "locacion": [],
            "locacion_id": [],
            "name_vendedor": [],
            "tipo_vendedor": [],
            "id_vendedor": [],
            "enlace": []
        }
        
    def _get_dataset(self):
        return self._dataset
    
    def _append_data(self, item, fecha_extraccion, enlace):
        self._dataset["titulo_marketplace"].append(item.get('marketplace_listing_title'))
        self._dataset["tiempo_creacion"].append(item.get('creation_time'))
        self._dataset["disponible"].append(item.get('is_live'))
        self._dataset["vendido"].append(item.get('is_sold'))
        self._dataset["cantidad"].append(item.get('listing_inventory_type'))
        self._dataset["name_vendedor"].append(item.get('story').get('actors')[0].get('name'))
        self._dataset["tipo_vendedor"].append(item.get('story').get('actors')[0]['__typename'])
        self._dataset["id_vendedor"].append(item.get('story').get('actors')[0]['id'])
        self._dataset["locacion_id"].append(item.get('location_vanity_or_id'))
        self._dataset["latitud"].append(item.get('location', {}).get('latitude'))
        self._dataset["longitud"].append(item.get('location', {}).get('longitude'))
        self._dataset["precio"].append(item.get('listing_price', {}).get('amount'))
        self._dataset["tipo_moneda"].append(item.get('listing_price', {}).get('currency'))
        self._dataset["amount_with_concurrency"].append(item.get('listing_price', {}).get('amount_with_offset_in_currency'))
        self._dataset["tipo_delivery"].append(item.get('delivery_types', [None])[0])
        self._dataset["delivery_data"].append(item.get("delivery_data", {}).get('carrier'))
        self._dataset["delivery_direccion"].append(item.get("delivery_data", {}).get('delivery_address'))
        self._dataset["descripcion"].append(item.get('redacted_description', {}).get('text'))
        self._dataset["fecha_union_vendedor"].append(item.get('marketplace_listing_seller', {}).get('join_time'))  
        data = item.get('location_text', {})
        if data:
            data = data.get('text')
        self._dataset["locacion"].append(data)
        self._dataset["Fecha Extraccion"].append(fecha_extraccion)
        self._dataset["enlace"].append(enlace)

In [4]:
class Tiempo:
    def __init__(self, start):
        self._hora_inicio = strftime("%H:%M:%S", localtime(start))
        log(INFO, f"Hora de inicio: {self._hora_inicio}")
        self._fecha = (datetime.now().date() - timedelta(days=1)).strftime('%d/%m/%Y')
        self._hora_fin = None
        self._cantidad = None
        self._tiempo = None
        self._productos_por_min = None
        self._enlace = None
        self._errores = None
        
    def _get_fecha(self):
        return self._fecha
    
    def _get_errores(self):
        return self._errores
    
    def _set_cantidad(self, cantidad):
        self._cantidad = cantidad
    
    def _set_errores(self, errores):
        self._errores = errores
    
    def _set_param_final(self, start):
        end = time()
        self._hora_fin = strftime("%H:%M:%S", localtime(end))
        log(INFO, f"Productos Extraídos: {self._cantidad}")
        log(INFO, f"Hora Fin: {self._hora_fin}")
        total = end - start
        self._tiempo = str(timedelta(seconds=total)).split(".")[0]
        self._productos_por_min = int(round(self._cantidad /(total / 60),0))

In [5]:
class ScraperFb:
    """Representa a un bot para hacer web scarping en fb marketplace.

    Attributes:
        driver (Object): Maneja un navegador para hacer web scraping
        wait (Object): Maneja el Tiempo de espera durante la ejecución del bot
    """
    
    def __init__(self, start):
        """Inicializa un objeto de tipo ScraperFb.

        Args:
            driver (Object): [Driver]
            wait (Object): [Wait]
        """
        log(INFO, "Inicializando scraper")
        self._tiempo = Tiempo(start)
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)
        self.driver = webdriver.Chrome(chrome_options=chrome_options,service=Service(ChromeDriverManager().install()))
        self.wait = WebDriverWait(self.driver, 10)
        self._errores = Errores()
        self._data = Dataset()

    def _get_data(self):
        return self._data
    
    def _get_errores(self):
        return self._errores
        
    def iniciar_sesion(self, url):
        """Inicia sesión en una página web usando un usuario y contraseña

        Args:
            url (str): [Url]
        """
        log(INFO, "Iniciando sesión")
        self.driver.get(url)
        self.driver.maximize_window()
        username = self.wait.until(EC.presence_of_element_located((By.ID, "email")))
        password = self.wait.until(EC.presence_of_element_located((By.ID, "pass")))
        username.clear()
        password.clear()
        username.send_keys(getenv('FB_USERNAME'))
        password.send_keys(getenv('FB_PASSWORD'))
        self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[name='login']"))).click()
        log(INFO, "Inicio de sesión con éxito")
        
    def mapear_datos(self, url):
        sleep(10)
        log(INFO, "Accediendo a la URL")
        self.driver.execute_script("window.open('about:blank', 'newtab');")
        self.driver.switch_to.window("newtab")
        self.driver.get(url)
        
        sleep(8)
        log(INFO, "Mapeando Publicaciones")
        ropa = self.driver.find_elements(By.XPATH, '//*[@class="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3"]')
        fecha_publicacion = fecha_extraccion = int(datetime.strptime(self._tiempo._get_fecha(),"%d/%m/%Y").timestamp())
        fecha_flag = fecha_extraccion + 86400
        i=0
        e=0
        del self.driver.requests
        
        while fecha_publicacion >= fecha_extraccion:
            log(INFO, f"Scrapeando item {i + 1}")
            
            try:
                try:
                    enlace = findall("(.*)\/\?", ropa[i].find_element(By.XPATH, ".//ancestor::a").get_attribute('href'))[0]
                except NoSuchElementException as error:
                    enlace = None
                    self._errores._append_error(error, enlace)
                ropa[i].click()
                sleep(5)
                for request in self.driver.requests:
                    if not request.response or 'graphql' not in request.url:
                        continue
                    
                    body = decode(request.response.body, request.response.headers.get('Content-Encoding', 'identity'))
                    decoded_body = body.decode('utf-8')
                    json_data = loads(decoded_body)
                    
                    if 'prefetch_uris_v2' not in json_data['extensions']:
                        continue

                    fecha_publicacion = json_data['data']['viewer']['marketplace_product_details_page']['target']['creation_time']
                    if fecha_publicacion < fecha_flag:
                        dato = json_data['data']['viewer']['marketplace_product_details_page']["target"]
                        log(INFO, f"{dato['marketplace_listing_title']}")
                        self._data._append_data(dato, self._tiempo._get_fecha(), enlace)
                        log(INFO, f"Item {i + 1} scrapeado con éxito")
                    break
                self.driver.execute_script("window.history.go(-1)");
                
            except (NoSuchElementException, ElementNotInteractableException, StaleElementReferenceException) as error:
                self._errores._append_error(error, enlace)
                e=e+1
                
            except (KeyError, JSONDecodeError) as error:
                self._errores._append_error(error, enlace)
                e=e+1
                self.driver.execute_script("window.history.go(-1)")
                
            except Exception as error:
                self._errores._append_error(error, enlace)
                e = e + 1
                log(CRITICAL, "Se detuvo inesperadamente el programa")
                log(CRITICAL, f"Causa:\n{error}")
                break
            i = i + 1
            if i == len(ropa):
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                sleep(7)
                ropa = self.driver.find_elements(By.XPATH, '//*[@class="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3"]')
            
            del self.driver.requests
            log(INFO, "-------------------------------------------------------------------")
            sleep(3)
        self._tiempo._set_errores(e)
        log(INFO, f"Se halló {e} errores")
        log(INFO, "Fin de la extraccion")
    
    def guardar_datos(self, dataset, filetype = "Data", folder="Data", filename="fb_data"):
        log(INFO, f"Guardando {filetype}")
        df_fb_mkp_ropa = DataFrame(dataset)
        if filetype == "Data":
            df_fb_mkp_ropa.drop(len(df_fb_mkp_ropa)-1, axis=0, inplace=True)
            cantidad = len(df_fb_mkp_ropa)
            self._tiempo._set_cantidad(cantidad)
        elif filetype == "Error":
            cantidad = self._tiempo._get_errores()
        else:
            return
        
        datetime_obj = datetime.strptime(self._tiempo._get_fecha(),"%d/%m/%Y")
        filepath = folder + "/" + datetime_obj.strftime('%d-%m-%Y') + "/"
        filename = filename + "_" + datetime_obj.strftime('%d%m%Y') + "_" + str(cantidad) + ".xlsx"
        if not path.exists(filepath):
            makedirs(filepath)
        df_fb_mkp_ropa.to_excel(filepath + filename, index = False)
        log(INFO, f"{filetype} Guardados Correctamente")
        
    def guardar_tiempos(self, filename, sheet_name, start):
        log(INFO, "Guardando tiempos")
        self._tiempo._set_param_final(start)
        tiempos = load_workbook(filename)
        header_exist = True
        if sheet_name not in [ws.title for ws in tiempos.worksheets]:
            tiempos.create_sheet(sheet_name)
            header_exist = False
        worksheet = tiempos[sheet_name]
        if not header_exist:
            worksheet.append(list(self._tiempo.__dict__.keys()))
        worksheet.append(list(self._tiempo.__dict__.values()))
        tiempos.save(filename)
        tiempos.close()
        log(INFO, "Tiempos Guardados Correctamente")

In [6]:
def config_log():
    seleniumLogger.setLevel(ERROR)
    urllibLogger.setLevel(ERROR)
    urllibLogger.propagate = False
    logger = getLogger('seleniumwire')
    logger.setLevel(ERROR)
    basicConfig(format='%(asctime)s %(message)s', level=INFO)

In [7]:
def main():
    # Formato para el debugger
    config_log()
    log(INFO, "Configurando Formato Básico del Debugger")
    
    # Cargar variables de entorno
    log(INFO, "Cargando Variables de entorno")
    load_dotenv()
    
    start = time()
    
    # Url base a scrapear
    url_base = 'https://www.facebook.com/'
    url_ropa = "https://www.facebook.com/marketplace/category/apparel/?sortBy=creation_time_descend&exact=false"
    
    # Parámetros para guardar la data extraída por el scraper
    data_filename = "fb_ropa"
    data_folder = data_type = "Data"
    
    # Parámetros para guardar la medición de la ejecución del scraper
    filename_tiempos = 'Tiempos.xlsx'
    sheet_tiempos = "Ropa"
    
    # Parámetros para guardar los errores durante la ejecución por el scraper
    error_filename = "fb_error"
    error_folder = error_type = "Error"
    
    scraper = ScraperFb(start)
    scraper.iniciar_sesion(url_base)
    scraper.mapear_datos(url_ropa)
    
    # Guardando la data extraída por el scraper
    scraper.guardar_datos(scraper._get_data()._get_dataset(), data_type, data_folder, data_filename)
    
    # Guardando los errores extraídos por el scraper
    scraper.guardar_datos(scraper._get_errores()._get_errores(), error_type, error_folder, error_filename)
    
    # Guardando los tiempos durante la ejecución del scraper
    scraper.guardar_tiempos(filename_tiempos, sheet_tiempos, start)
    
    log(INFO, "Programa ejecutado satisfactoriamente")

In [8]:
if __name__ == '__main__':
    main()

2023-01-10 09:29:14,917 Configurando Formato Básico del Debugger
2023-01-10 09:29:14,920 Cargando Variables de entorno
2023-01-10 09:29:14,923 Inicializando scraper
2023-01-10 09:29:14,924 Hora de inicio: 09:29:14
2023-01-10 09:29:14,925 ====== WebDriver manager ======
2023-01-10 09:29:15,884 Get LATEST chromedriver version for google-chrome 108.0.5359
2023-01-10 09:29:17,617 Driver [C:\Users\param\.wdm\drivers\chromedriver\win32\108.0.5359\chromedriver.exe] found in cache
2023-01-10 09:29:19,871 Iniciando sesión
2023-01-10 09:29:25,582 Inicio de sesión con éxito
2023-01-10 09:29:35,596 Accediendo a la URL
2023-01-10 09:29:55,266 Mapeando Publicaciones
2023-01-10 09:29:55,537 Scrapeando item 1
2023-01-10 09:30:00,896 -------------------------------------------------------------------
2023-01-10 09:30:03,912 Scrapeando item 2
2023-01-10 09:30:09,151 -------------------------------------------------------------------
2023-01-10 09:30:12,155 Scrapeando item 3
2023-01-10 09:30:17,467 -----

2023-01-10 09:37:50,217 Scrapeando item 57
2023-01-10 09:37:55,378 -------------------------------------------------------------------
2023-01-10 09:37:58,386 Scrapeando item 58
2023-01-10 09:38:03,631 -------------------------------------------------------------------
2023-01-10 09:38:06,636 Scrapeando item 59
2023-01-10 09:38:11,797 -------------------------------------------------------------------
2023-01-10 09:38:14,805 Scrapeando item 60
2023-01-10 09:38:19,967 -------------------------------------------------------------------
2023-01-10 09:38:22,970 Scrapeando item 61
2023-01-10 09:38:28,155 -------------------------------------------------------------------
2023-01-10 09:38:31,170 Scrapeando item 62
2023-01-10 09:38:36,389 -------------------------------------------------------------------
2023-01-10 09:38:39,403 Scrapeando item 63
2023-01-10 09:38:44,597 -------------------------------------------------------------------
2023-01-10 09:38:47,605 Scrapeando item 64
2023-01-10 0

2023-01-10 09:46:32,146 Scrapeando item 118
2023-01-10 09:46:37,412 -------------------------------------------------------------------
2023-01-10 09:46:40,426 Scrapeando item 119
2023-01-10 09:46:45,715 -------------------------------------------------------------------
2023-01-10 09:46:48,731 Scrapeando item 120
2023-01-10 09:46:53,970 -------------------------------------------------------------------
2023-01-10 09:46:56,986 Scrapeando item 121
2023-01-10 09:47:02,212 -------------------------------------------------------------------
2023-01-10 09:47:05,223 Scrapeando item 122
2023-01-10 09:47:10,475 -------------------------------------------------------------------
2023-01-10 09:47:13,479 Scrapeando item 123
2023-01-10 09:47:18,748 -------------------------------------------------------------------
2023-01-10 09:47:21,758 Scrapeando item 124
2023-01-10 09:47:26,968 -------------------------------------------------------------------
2023-01-10 09:47:29,981 Scrapeando item 125
2023

2023-01-10 09:55:13,442 -------------------------------------------------------------------
2023-01-10 09:55:16,456 Scrapeando item 179
2023-01-10 09:55:21,679 -------------------------------------------------------------------
2023-01-10 09:55:24,679 Scrapeando item 180
2023-01-10 09:55:29,885 -------------------------------------------------------------------
2023-01-10 09:55:32,891 Scrapeando item 181
2023-01-10 09:55:38,136 -------------------------------------------------------------------
2023-01-10 09:55:41,149 Scrapeando item 182
2023-01-10 09:55:46,372 -------------------------------------------------------------------
2023-01-10 09:55:49,381 Scrapeando item 183
2023-01-10 09:55:54,594 -------------------------------------------------------------------
2023-01-10 09:55:57,600 Scrapeando item 184
2023-01-10 09:56:02,878 -------------------------------------------------------------------
2023-01-10 09:56:05,884 Scrapeando item 185
2023-01-10 09:56:11,139 ------------------------

2023-01-10 10:03:45,234 Scrapeando item 239
2023-01-10 10:03:50,472 -------------------------------------------------------------------
2023-01-10 10:03:53,486 Scrapeando item 240
2023-01-10 10:04:05,813 -------------------------------------------------------------------
2023-01-10 10:04:08,829 Scrapeando item 241
2023-01-10 10:04:14,070 -------------------------------------------------------------------
2023-01-10 10:04:17,072 Scrapeando item 242
2023-01-10 10:04:22,286 -------------------------------------------------------------------
2023-01-10 10:04:25,298 Scrapeando item 243
2023-01-10 10:04:30,625 -------------------------------------------------------------------
2023-01-10 10:04:33,634 Scrapeando item 244
2023-01-10 10:04:38,891 -------------------------------------------------------------------
2023-01-10 10:04:41,893 Scrapeando item 245
2023-01-10 10:04:47,140 -------------------------------------------------------------------
2023-01-10 10:04:50,156 Scrapeando item 246
2023

...

2023-01-10 10:46:16,304 Flare pant beige talla SML material scuba
2023-01-10 10:46:16,305 Item 535 scrapeado con éxito
2023-01-10 10:46:16,352 -------------------------------------------------------------------
2023-01-10 10:46:19,361 Scrapeando item 536
2023-01-10 10:46:24,587 Camisas Zara niño
2023-01-10 10:46:24,589 Item 536 scrapeado con éxito
2023-01-10 10:46:24,656 -------------------------------------------------------------------
2023-01-10 10:46:27,668 Scrapeando item 537
2023-01-10 10:46:32,843 Short NIKE talla L (34-36) en perfecto estado
2023-01-10 10:46:32,845 Item 537 scrapeado con éxito
2023-01-10 10:46:32,895 -------------------------------------------------------------------
2023-01-10 10:46:35,911 Scrapeando item 538
2023-01-10 10:46:41,128 Falda short
2023-01-10 10:46:41,130 Item 538 scrapeado con éxito
2023-01-10 10:46:41,175 -------------------------------------------------------------------
2023-01-10 10:46:44,177 Scrapeando item 539
2023-01-10 10:46:49,367 Short 

2023-01-10 10:51:03,576 -------------------------------------------------------------------
2023-01-10 10:51:06,592 Scrapeando item 569
2023-01-10 10:51:11,813 Remate airmax
2023-01-10 10:51:11,814 Item 569 scrapeado con éxito
2023-01-10 10:51:11,854 -------------------------------------------------------------------
2023-01-10 10:51:14,858 Scrapeando item 570
2023-01-10 10:51:20,110 Short LA MARTINA original (32)
2023-01-10 10:51:20,112 Item 570 scrapeado con éxito
2023-01-10 10:51:20,156 -------------------------------------------------------------------
2023-01-10 10:51:23,167 Scrapeando item 571
2023-01-10 10:51:28,399 Biker cuerina
2023-01-10 10:51:28,400 Item 571 scrapeado con éxito
2023-01-10 10:51:28,438 -------------------------------------------------------------------
2023-01-10 10:51:31,443 Scrapeando item 572
2023-01-10 10:51:36,668 Polos hombre
2023-01-10 10:51:36,669 Item 572 scrapeado con éxito
2023-01-10 10:51:36,725 ----------------------------------------------------

2023-01-10 10:55:52,794 -------------------------------------------------------------------
2023-01-10 10:55:55,808 Scrapeando item 603
2023-01-10 10:56:01,010 Zapatos Cat originales C/P tall 41
2023-01-10 10:56:01,013 Item 603 scrapeado con éxito
2023-01-10 10:56:01,064 -------------------------------------------------------------------
2023-01-10 10:56:04,078 Scrapeando item 604
2023-01-10 10:56:09,323 Zapatillas adidas de mujer talla 40 , estado 9 de 10
2023-01-10 10:56:09,324 Item 604 scrapeado con éxito
2023-01-10 10:56:09,388 -------------------------------------------------------------------
2023-01-10 10:56:12,402 Scrapeando item 605
2023-01-10 10:56:17,618 Remate zapatillas
2023-01-10 10:56:17,620 Item 605 scrapeado con éxito
2023-01-10 10:56:17,667 -------------------------------------------------------------------
2023-01-10 10:56:20,675 Scrapeando item 606
2023-01-10 10:56:25,909 Nike tiempo just do it.
2023-01-10 10:56:25,910 Item 606 scrapeado con éxito
2023-01-10 10:56:2

2023-01-10 11:00:49,028 Item 636 scrapeado con éxito
2023-01-10 11:00:49,100 -------------------------------------------------------------------
2023-01-10 11:00:52,112 Scrapeando item 637
2023-01-10 11:00:57,360 Abrigo Guess original, talla s
2023-01-10 11:00:57,361 Item 637 scrapeado con éxito
2023-01-10 11:00:57,408 -------------------------------------------------------------------
2023-01-10 11:01:00,414 Scrapeando item 638
2023-01-10 11:01:05,632 Remató Zara
2023-01-10 11:01:05,633 Item 638 scrapeado con éxito
2023-01-10 11:01:05,680 -------------------------------------------------------------------
2023-01-10 11:01:08,695 Scrapeando item 639
2023-01-10 11:01:13,895 Manguitas tejidas
2023-01-10 11:01:13,896 Item 639 scrapeado con éxito
2023-01-10 11:01:13,964 -------------------------------------------------------------------
2023-01-10 11:01:16,978 Scrapeando item 640
2023-01-10 11:01:22,204 Nike
2023-01-10 11:01:22,206 Item 640 scrapeado con éxito
2023-01-10 11:01:22,261 -----

2023-01-10 11:05:41,239 Scrapeando item 671
2023-01-10 11:05:46,495 ULTIMOS MODELOS DISPONIBLES EN SANDALIAS IMPORTADAS ORIGINALES MODELO EVA
2023-01-10 11:05:46,496 Item 671 scrapeado con éxito
2023-01-10 11:05:46,544 -------------------------------------------------------------------
2023-01-10 11:05:49,557 Scrapeando item 672
2023-01-10 11:05:54,772 Remato pantalón jeans Américano  SNIKER talla 38 Nuevo
2023-01-10 11:05:54,773 Item 672 scrapeado con éxito
2023-01-10 11:05:54,835 -------------------------------------------------------------------
2023-01-10 11:05:57,850 Scrapeando item 673
2023-01-10 11:06:03,050 Conjunto estilo sastre 🪩
2023-01-10 11:06:03,051 Item 673 scrapeado con éxito
2023-01-10 11:06:03,106 -------------------------------------------------------------------
2023-01-10 11:06:06,109 Scrapeando item 674
2023-01-10 11:06:11,369 CASACAS DENIN
2023-01-10 11:06:11,369 Item 674 scrapeado con éxito
2023-01-10 11:06:11,431 ------------------------------------------------